## huggingface transformers llama代码
transformers/src/transformers/models/llama/modeling_llama.py
- PrTrainedModel是所有模型的基类
- LlamaPreTrainedModel是PrTrainedModel的子类
    - LlamaPreTrainedModel 重写了 _init_weights 和 _set_gradient_checkpoint 这两个方法
- LlamaModel是PrTrainedModel的子类

## 1、LlamaModel

1、初始化方法

In [ ]:
def __init__(self, config: LlamaConfig):  # 接受一个LlamaConfig类型的参数config，用于配置模型的各种参数
    super().__init__(config)  # 因为继承了LlamaPreTrainedModel，调用父类的构造函数
    self.padding_idx = config.pad_token_id  # 用于指定填充标记的索引
    self.vocab_size = config.vocab_size  # 词表大小

    self.embed_tokens = nn.Embedding(
        config.vocab_size, config.hidden_size, self.padding_idx)  # 初始化Embedding层
    self.layers = nn.ModuleList(
        [LlamaDecoderLayer(config, layer_idx)
         for layer_idx in range(config.num_hidden_layers)]
    )  # 初始化解码器层
    self._use_sdpa = config._attn_implementation == "sdpa"
    self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
    self.norm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)  # 归一化层

    self.gradient_checkpointing = False  # 模型训练中的内存优化技术
    # Initialize weights and apply final processing
    self.post_init()  # 完成一些初始化和准备检查的代码

2、input_embeddings

In [ ]:
# 获取self.embed_tokens
def get_input_embeddings(self):
    return self.embed_tokens

# 设置self.embed_tokens
def set_input_embeddings(self, value):
    self.embed_tokens = value

## 2、LlamaDecoderLayer

1、初始化方法

In [ ]:
def __init__(self, config: LlamaConfig):
    super().__init__()
    self.hidden_size = config.hidden_size
    self.self_attn = LlamaAttention(config=config)
    self.mlp = LlamaMLP(config)
    self.input_layernorm = LlamaRMSNorm(
        config.hidden_size, eps=config.rms_norm_eps)
    self.post_attention_layernorm = LlamaRMSNorm(
        config.hidden_size, eps=config.rms_norm_eps)

在初始化方法中，定义了解码器层的结构。它接受一个 config 参数，其中包含了解码器层的配置信息，如隐藏层大小、RMS归一化的epsilon等。
初始化方法首先调用了 super().__init__() 来初始化父类 nn.Module；
然后创建以下子模块：
self.self_attn：这是一个自注意力机制模块，通常包含多头注意力机制。
self.mlp：这是一个前馈神经网络（多层感知器）模块。
self.input_layernorm：用于输入数据的RMS归一化层。
self.post_attention_layernorm：用于自注意力输出后的RMS归一化层。


2、forward方法

In [ ]:
def forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_value: Optional[Tuple[torch.Tensor]] = None,
    output_attentions: Optional[bool] = False,
    use_cache: Optional[bool] = False,
) -> Tuple[torch.FloatTensor, Optional[Tuple[torch.FloatTensor, torch.FloatTensor]]]:
    residual = hidden_states

    hidden_states = self.input_layernorm(hidden_states)

    # Self Attention
    hidden_states, self_attn_weights, present_key_value = self.self_attn(
        hidden_states=hidden_states,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_value=past_key_value,
        output_attentions=output_attentions,
        use_cache=use_cache,
    )
    hidden_states = residual + hidden_states

    # Fully Connected
    residual = hidden_states
    hidden_states = self.post_attention_layernorm(hidden_states)
    hidden_states = self.mlp(hidden_states)
    hidden_states = residual + hidden_states

    outputs = (hidden_states,)

    if output_attentions:
        outputs += (self_attn_weights,)

    if use_cache:
        outputs += (present_key_value,)

    return outputs

输入参数:
hidden_states（torch.Tensor）：输入到解码器层的张量，具有形状 (batch, seq_len, embed_dim)，其中 batch 表示批量大小，seq_len 表示序列长度，embed_dim 表示嵌入维度。
attention_mask（Optional[torch.Tensor]，可选）：注意力掩码张量，具有形状 (batch, 1, tgt_len, src_len)，其中 tgt_len 表示目标序列长度，src_len 表示源序列长度。它用于指示哪些位置需要注意力，哪些位置需要忽略。这通常包括对输入序列中的填充位置进行掩码。
position_ids（Optional[torch.LongTensor]，可选）：位置编码张量，用于表示输入序列中每个位置的位置信息。它的形状应该与 hidden_states 相同。
past_key_value（Optional[Tuple[torch.Tensor]]，可选）：过去的键值对状态，用于加速解码过程。这是一个元组，包含两个张量，分别是过去的键（key）和过去的值（value）。如果不提供，可以设置为None。
output_attentions（Optional[bool]，可选）：一个布尔值，指示是否返回自注意力权重。如果为True，将在输出元组中包含自注意力权重，否则不包含。
use_cache（Optional[bool]，可选）：一个布尔值，指示是否使用过去的键值对状态以加速解码。如果为True，将在输出元组中包含 present_key_value，否则不包含。
残差连接 (Residual Connection):
开始时，将输入 hidden_states 复制到 residual 变量中，以便在后续步骤中实现残差连接。
输入归一化 (Input Layer Normalization):
将输入 hidden_states 传递给输入归一化层 self.input_layernorm 进行归一化操作。这有助于稳定训练过程，确保输入的均值和方差适当地缩放。
自注意力 (Self-Attention):
使用自注意力机制 self.self_attn 处理已归一化的 hidden_states。
自注意力机制通常包括多头自注意力，用于捕捉输入序列中不同位置之间的关系。
在执行自注意力操作后，得到新的 hidden_states，以及可能的自注意力权重 self_attn_weights 和过去的键值对状态 present_key_value
残差连接 (Residual Connection):
将自注意力的输出与初始的 residual 相加，实现残差连接。这有助于信息流动，避免梯度消失问题。
输出归一化 (Layer Normalization after Attention):
再次将残差连接后的 hidden_states 传递给输出归一化层 self.post_attention_layernorm 进行归一化操作。
前馈神经网络 (MLP):
将经过输出归一化的 hidden_states 传递给前馈神经网络 self.mlp 进行处理。前馈神经网络通常包括多层全连接层。
残差连接 (Residual Connection):
再次将前馈神经网络的输出与之前的 hidden_states 相加，再次应用残差连接。
构建输出:
最终的输出是新的 hidden_states，存储在元组 outputs 中。
如果 output_attentions 为True，还会将自注意力权重 self_attn_weights 添加到 outputs 中。
如果 use_cache 为True，还会将过去的键值对状态 present_key_value 添加到 outputs 中。

## 4、LlamaMLP

In [ ]:
class LlamaMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.intermediate_size = config.intermediate_size
        self.gate_proj = nn.Linear(
            self.hidden_size, self.intermediate_size, bias=False)
        self.up_proj = nn.Linear(
            self.hidden_size, self.intermediate_size, bias=False)
        self.down_proj = nn.Linear(
            self.intermediate_size, self.hidden_size, bias=False)
        self.act_fn = ACT2FN[config.hidden_act]

    def forward(self, x):
        if self.config.pretraining_tp > 1:
            slice = self.intermediate_size // self.config.pretraining_tp
            gate_proj_slices = self.gate_proj.weight.split(slice, dim=0)
            up_proj_slices = self.up_proj.weight.split(slice, dim=0)
            down_proj_slices = self.down_proj.weight.split(slice, dim=1)

            gate_proj = torch.cat(
                [F.linear(x, gate_proj_slices[i]) for i in range(self.config.pretraining_tp)], dim=-1
            )
            up_proj = torch.cat([F.linear(x, up_proj_slices[i])
                                for i in range(self.config.pretraining_tp)], dim=-1)

            intermediate_states = (self.act_fn(
                gate_proj) * up_proj).split(slice, dim=2)
            down_proj = [
                F.linear(intermediate_states[i], down_proj_slices[i]) for i in range(self.config.pretraining_tp)
            ]
            down_proj = sum(down_proj)
        else:
            down_proj = self.down_proj(self.act_fn(
                self.gate_proj(x)) * self.up_proj(x))

        return down_proj

1、初始化方法
- gate_proj、up_proj 和 down_proj 是三个线性变换层，用于将输入张量 x 映射到不同的表示空间。它们分别用于产生门控信号、升维和降维操作。
- act_fn 是一个激活函数，根据配置中指定的激活函数类型选择相应的激活函数。ACT2FN 是一个映射，将激活函数名称映射到PyTorch中的实际激活函数。  

2、forward
- 如果 config.pretraining_tp 大于1，表示需要将MLP的权重参数分成多个块进行预训练。在这种情况下，输入张量 x 和三个线性层的权重参数都会被分成多个块;
- 对门控信号和升维操作的输出进行分块线性变换，以得到中间状态；
- 门控信号和升维操作的输出逐元素相乘后，过激活函数，然后再进行降维操作；
- 最后，将中间状态的块求和，得到最终的输出 down_proj。
这三个线性层在MLP中的作用如下：
1. 门控信号 (gate_proj)： 门控信号层的作用是引入门控机制，允许模型选择性地激活或抑制输入特征的不同部分。这种选择性激活可以使模型更加灵活地学习到输入数据的复杂关系，有助于提高模型的表达能力。门控信号通常经过激活函数后，用于控制升维操作的输入；
2. 升维操作 (up_proj)： 升维操作通过将输入特征映射到一个更高维度的表示空间，有助于模型更好地捕捉输入特征之间的非线性关系。这个操作可以增加模型的容量，使其能够表示更复杂的函数。升维操作通常是一个线性变换，它将输入特征进行线性组合，并通过激活函数引入非线性；
3. 降维操作 (down_proj)： 降维操作与升维操作相反，它将高维度的特征映射回原始的隐藏状态维度。这个操作的目的是将经过门控信号和升维操作处理后的特征重新投影到原始表示空间，以保留重要信息并减少冗余。这有助于控制模型的复杂度，减少计算成本，同时保持模型的表达能力。

## 5、LlamaRMSNorm

In [ ]:
class LlamaRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        LlamaRMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype  # 获取hidden_states的数据类型
        hidden_states = hidden_states.to(torch.float32)  # 转换为float32类型
        variance = hidden_states.pow(2).mean(-1, keepdim=True)  # 计算方差
        hidden_states = hidden_states * \
            torch.rsqrt(variance + self.variance_epsilon)  # 计算归一化后的hidden_states
        return self.weight * hidden_states.to(input_dtype)  # 将标准化后的张量和可学习的权重参数相乘

## 6、LlamaRotaryEmbedding
位置编码方法


1、初始化方法

In [ ]:
def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
    super().__init__()

    self.dim = dim
    self.max_position_embeddings = max_position_embeddings
    self.base = base
    inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2).float().to(device) / self.dim))
    self.register_buffer("inv_freq", inv_freq, persistent=False)

    # Build here to make `torch.jit.trace` work.
    self._set_cos_sin_cache(
        seq_len=max_position_embeddings, device=self.inv_freq.device, dtype=torch.get_default_dtype()
    )

def _set_cos_sin_cache(self, seq_len, device, dtype):
    self.max_seq_len_cached = seq_len
    t = torch.arange(self.max_seq_len_cached, device=device, dtype=self.inv_freq.dtype)

    freqs = torch.einsum("i,j->ij", t, self.inv_freq)
    # Different from paper, but it uses a different permutation in order to obtain the same calculation
    emb = torch.cat((freqs, freqs), dim=-1)
    self.register_buffer("cos_cached", emb.cos()[None, None, :, :].to(dtype), persistent=False)
    self.register_buffer("sin_cached", emb.sin()[None, None, :, :].to(dtype), persistent=False)

2、forward

In [ ]:
def forward(self, x, seq_len=None):
    # x: [bs, num_attention_heads, seq_len, head_size]
    if seq_len > self.max_seq_len_cached:
        self._set_cos_sin_cache(
                seq_len=seq_len, device=x.device, dtype=x.dtype)

    return (
        self.cos_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
        self.sin_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
    )

## 7、LlamaLinearScalingRotaryEmbedding
在原有RoPE的基础上添加了线性缩放的功能

In [ ]:
def _set_cos_sin_cache(self, seq_len, device, dtype):
    self.max_seq_len_cached = seq_len
    t = torch.arange(self.max_seq_len_cached,
                     device=device, dtype=self.inv_freq.dtype)
    t = t / self.scaling_factor

    freqs = torch.einsum("i,j->ij", t, self.inv_freq)
    # Different from paper, but it uses a different permutation in order to obtain the same calculation
    emb = torch.cat((freqs, freqs), dim=-1)
    self.register_buffer("cos_cached", emb.cos()[
        None, None, :, :].to(dtype), persistent=False)
    self.register_buffer("sin_cached", emb.sin()[
        None, None, :, :].to(dtype), persistent=False)